In [1]:
%load_ext autoreload

In [2]:
# 1. Imports
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import re
import time
import pyautogui
import matplotlib
# import cv2

# import scipy
from scipy import interpolate
from scipy import signal

# import skimage
from skimage import io, filters, exposure, measure, transform, util
from scipy.signal import find_peaks, savgol_filter
from scipy.optimize import linear_sum_assignment

# 2. Pandas settings
pd.set_option('mode.chained_assignment',None)

# 3. Plot settings
# Here we use this mode because displaying images 
# in new windows is more convenient for this code.
%matplotlib qt 
# To switch back to inline display, use : 
# %matplotlib widget or %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})

SMALLER_SIZE = 8
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# 4. Other settings
# These regex are used to correct the stupid date conversions done by Excel
dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [3]:
%autoreload 1

%aimport BeadTracker

In [4]:
experimentalDataDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//ExperimentalData"

expDf = BeadTracker.getExperimentalConditions(experimentalDataDir, save = False)

expDf.tail()

Extracted a table with 57 lines and 24 columns.
optical index correction already in Float64 type.
dates corrected


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,multi image Z step,loop structure,comments,manipID
52,21-09-09,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.154,...,0,5,3_40,1s,True,3,500,170_133_1,<NA>,21-09-09_M2
53,21-10-18,M1,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.114,...,0,5,5_40,1s,False,3,500,169_133,<NA>,21-10-18_M1
54,21-10-18,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.100,...,0,1,1_10,1s,False,3,500,169_133,<NA>,21-10-18_M2
55,21-10-25,M1,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.140,...,0,1,1_10,1s,False,3,500,169_133,<NA>,21-10-25_M1
56,21-10-25,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.100,...,0,5,5_40,1s,False,3,500,169_133,<NA>,21-10-25_M2


In [5]:
def main(dates, manips, wells, cells, depthoNames, methodT, factorT, redoAllSteps = False):
    
    start = time.time()
    
    # 0. Load different data sources & Preprocess : fluo, black images, sort slices (ct/ramp ; down/middle/up)
    # Make list of files to analyse
    imagesToAnalyse = []
    imagesToAnalyse_Paths = []
    if not isinstance(dates, str):
        rawDirList = [os.path.join(rawDataDir, d) for d in dates]
    else:
        rawDirList = [os.path.join(rawDataDir, dates)]
    for rd in rawDirList:
        fileList = os.listdir(rd)
        for f in fileList:
            if BeadTracker.isFileOfInterest(f, manips, wells, cells):
                imagesToAnalyse.append(f)
                imagesToAnalyse_Paths.append(os.path.join(rd, f))    

    # Begining of the MAIN LOOP
    
    for i in range(len(imagesToAnalyse)): 
        f, fP = imagesToAnalyse[i], imagesToAnalyse_Paths[i]
        manipID = BeadTracker.findInfosInFileName(f, 'manipID')
        cellID = BeadTracker.findInfosInFileName(f, 'cellID')
        
        print('\n')
        print('Analysis of file {:.0f}/{:.0f} : {}'.format(i+1, len(imagesToAnalyse), f))
        print('Loading image and experimental data...')
        
        # Load exp data
        if manipID not in expDf['manipID'].values:
            print('Error! No experimental data found for: ' + manipID)
            bug
        else:
            expDf_line = expDf.loc[expDf['manipID'] == manipID]
            manipDict = {}
            for c in expDf_line.columns.values:
                manipDict[c] = expDf_line[c].values[0]
        
#         # Extract some data from exp data
#         StrBeadTypes = str(manipDict['bead type'])
#         if '_' in StrBeadTypes:
#             beadTypes = [bT for bT in StrBeadTypes.split('_')]
#         else:
#             beadTypes = [StrBeadTypes]
        
#         StrBeadDiameters = str(manipDict['bead diameter'])
#         if '_' in StrBeadDiameters:
#             beadDiameters = [int(bD) for bD in StrBeadDiameters.split('_')]
#         else:
#             beadDiameters = [int(StrBeadDiameters)]
            
#         dictBeadDiameters = {}
#         for k in range(len(beadTypes)):
#             dictBeadDiameters[beadTypes[k]] = beadDiameters[k]
    
    
        # Load image and init PTL
        I = io.imread(fP) # Approx 0.5s per image
        PTL = BeadTracker.PincherTimeLapse(I, cellID, manipDict, NB = 2)
    
        # Load field file
        fieldFilePath = fP[:-4] + '_Field.txt'
        fieldCols = ['B_set', 'T_abs', 'B', 'Z']
        fieldDf = pd.read_csv(fieldFilePath, sep = '\t', names = fieldCols)
        
        # Check if a log file exists and load it if required
        logFilePath = fP[:-4] + '_LogPY.txt'
        logFileImported = False
        if redoAllSteps:
            pass
        elif os.path.isfile(logFilePath):
            PTL.importLog(logFilePath)
            PTL.dictLog['UILog'] = PTL.dictLog['UILog'].astype(str)
            logFileImported = True
            
        print('OK!')
        
        print('Pretreating the image...')
        
        # Detect fluo & black images
        PTL.checkIfBlackFrames()
        PTL.saveFluoAside()
        if not logFileImported:
            pass # I may change the organization of this part later
        
        # Sort slices
        if not logFileImported:
            if 'R40' or 'thickness5mT' in f:
                PTL.determineFramesStatus_R40()
        PTL.saveLog(display = False, save = (not logFileImported), path = logFilePath)
        
        # Import or determine global threshold
        MDpath = fP[:-4] + '_MetaDataPY.txt'
        if redoAllSteps:
            PTL.uiThresholding(method = methodT, factorT = factorT)
        else:
            try:
                PTL.threshold = PTL.readMetaData(MDpath, 'threshold')
            except:
                PTL.uiThresholding(method = methodT, factorT = factorT) # Approx 3s per image
#         PTL.testThresholding()

        # Save some metadata
        PTL.saveMetaData(MDpath)
        
        # Create list of Frame objects
        PTL.makeFramesList()
        
        print('OK!')
        
    # 1. Detect beads
    
        print('Detecting all the bead objects...')
        Td = time.time()
        
        # Check if a _Results.txt exists and import it if so
        resFilePath = fP[:-4] + '_ResultsPY.txt'
        resFileImported = False
        if redoAllSteps:
            pass
        elif os.path.isfile(resFilePath):
            PTL.importBeadsDetectResult(resFilePath)
            resFileImported = True
    
        # Dectect the beads and create the BeadsDetectResult dataframe [if no file has been loaded before] 
        # OR input the results in each Frame objects [if the results have been loaded at the previous step]
        PTL.detectBeads(resFileImported, display = 0)
        
        # Save the new results if necessary
        if not resFileImported:
            PTL.saveBeadsDetectResult(path=resFilePath)
            
        print('OK! dT = {:.3f}'.format(time.time()-Td))

    # 2. Make trajectories for beads of interest
    
        print('Tracking the beads of interest...')
        Tt = time.time()
        
        # One of the main steps ! The tracking of the beads happens here !
        issue = PTL.buildTrajectories() 
        if issue == 'Bug':
            continue
        else:
            pass
        
        # Save the user inputs
        PTL.saveLog(display = 0, save = True, path = logFilePath)
        
        print('OK! dT = {:.3f}'.format(time.time()-Tt))
    
    # 3. Qualify - Detect boi sizes and neighbours
        # Detect or infer Boi sizes in the first image
        if len(PTL.beadTypes) == 1:
            if 'M450' in PTL.beadTypes[0]:
                D = 4.5
            elif 'M270' in PTL.beadTypes[0]:
                D = 2.7
            for B in PTL.listFrames[0].listBeads:
                B.D = D
        else:
            PTL.listFrames[0].detectDiameter(plot = 0)
            
        # Propagate it across the trajectories
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            B0 = traj.dict['Bead'][0]
            D = B0.D
            traj.D = D
            for B in traj.dict['Bead']:
                B.D = D
        
        # Detect neighbours in every first image of loop TBC !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # !!!!!!
        
#         iL = 0 # index to count the loops
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            beadType = ''
            if len(PTL.beadTypes) == 1:
                beadType = PTL.beadTypes[0] # M270 or M450
            else:
                beadType = 'detect'
            traj.detectNeighbours(frequency = PTL.loop_totalSize, beadType = beadType)
            

    
        
    # 4. Compute dz
        # Import depthographs
        HDZfactor = 10
        
        if len(PTL.beadTypes) == 1:
            depthoPath = os.path.join(depthoDir, depthoNames)
#             depthoExist = os.path.exists(depthoPath+'_Deptho.tif')
            deptho = io.imread(depthoPath+'_Deptho.tif')
            depthoMetadata = pd.read_csv(depthoPath+'_Metadata.csv', sep=';')
            depthoStep = depthoMetadata.loc[0,'step']
            depthoZFocus = depthoMetadata.loc[0,'focus']
            
            # increase the resolution of the deptho with interpolation
            nX, nZ = deptho.shape[1], deptho.shape[0]
            XX, ZZ = np.arange(0, nX, 1), np.arange(0, nZ, 1)
            fd = interpolate.interp2d(XX, ZZ, deptho, kind='cubic')
            ZZ_HD = np.arange(0, nZ, 1/HDZfactor)
            depthoHD = fd(XX, ZZ_HD)
            depthoStepHD = depthoStep/HDZfactor
            depthoZFocus = depthoZFocus*HDZfactor
            #
            for iB in range(PTL.NB):
                traj = PTL.listTrajectories[iB]
                traj.deptho = depthoHD
                traj.depthoPath = depthoPath
                traj.depthoStep = depthoStepHD
                traj.depthoZFocus = depthoZFocus
        
        if len(PTL.beadTypes) > 1:
            for dN in depthoNames:
                depthoPath = os.path.join(depthoDir, dN)
                deptho = io.imread(depthoPath+'_Deptho.tif')
                depthoMetadata = pd.read_csv(depthoPath+'_Metadata.csv', sep=';')
                depthoStep = depthoMetadata.loc[0,'step']
                depthoZFocus = depthoMetadata.loc[0,'focus']
                
                # increase the resolution of the deptho with interpolation
                nX, nZ = deptho.shape[1], deptho.shape[0]
                XX, ZZ = np.arange(0, nX, 1), np.arange(0, nZ, 1)
                fd = interpolate.interp2d(XX, ZZ, deptho, kind='cubic')
                ZZ_HD = np.arange(0, nZ, 1/HDZfactor)
                depthoHD = fd(XX, ZZ_HD)
                depthoStepHD = depthoStep/HDZfactor
                depthoZFocusHD = depthoZFocus*HDZfactor
                #
                if 'M450' in dN:
                    depthoD = 4.5
                elif 'M270' in dN:
                    depthoD = 2.7
                for iB in range(PTL.NB):
                    traj = PTL.listTrajectories[iB]
                    if traj.D == depthoD:
                        traj.deptho = depthoHD
                        traj.depthoPath = depthoPath
                        traj.depthoStep = depthoStepHD
                        traj.depthoZFocus = depthoZFocusHD
            
        # Compute z for each traj
        for iB in range(PTL.NB):
            np.set_printoptions(threshold=np.inf)
            
            print('Computing Z in traj  {:.0f}...'.format(iB+1))
            Tz = time.time()
            traj = PTL.listTrajectories[iB]
            traj.computeZ(plot = 0)
            print('OK! dT = {:.3f}'.format(time.time()-Tz))
            
        # Keep only the best std data in the trajectories
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            traj.keepBestStdOnly()
    
    
    # 5. Define pairs and compute dx, dy
        
        print('Computing distances...')
        if PTL.NB == 2:
            traj1 = PTL.listTrajectories[0]
            traj2 = PTL.listTrajectories[1]
            nT = traj1.nT
            
            # Create a dict to prepare the export of the results
            timeSeries = {
                'idxCompression' : np.zeros(nT),
                'T' : np.zeros(nT),
                'Tabs' : np.zeros(nT),
                'B' : np.zeros(nT),
                'F' : np.zeros(nT),
                'dx' : np.zeros(nT),
                'dy' : np.zeros(nT),
                'dz' : np.zeros(nT),
                'D2' : np.zeros(nT),
                'D3' : np.zeros(nT)
            }

            # Input common values:
            T0 = fieldDf['T_abs'].values[0]/1000
            timeSeries['idxCompression'] = traj1.dict['idxCompression']
            timeSeries['Tabs'] = (fieldDf['T_abs'][traj1.dict['iField']])/1000
            timeSeries['T'] = timeSeries['Tabs'].values - T0*np.ones(nT)
            timeSeries['B'] = fieldDf['B_set'][traj1.dict['iField']].values
            timeSeries['B'] *= PTL.MagCorrFactor

            # Compute distances
            timeSeries['dx'] = (traj2.dict['X'] - traj1.dict['X'])/PTL.scale
            timeSeries['dy'] = (traj2.dict['Y'] - traj1.dict['Y'])/PTL.scale
            timeSeries['D2'] = (timeSeries['dx']**2 +  timeSeries['dy']**2)**0.5
            
            timeSeries['dz'] = (traj2.dict['Zr']*traj2.depthoStep - traj1.dict['Zr']*traj1.depthoStep)/1000
            timeSeries['dz'] *= PTL.OptCorrFactor
            timeSeries['D3'] = (timeSeries['D2']**2 +  timeSeries['dz']**2)**0.5
            
            
            #print('\n\n* timeSeries:\n')
            #print(timeSeries_DF[['T','B','F','dx','dy','dz','D2','D3']])
            print('OK!')
            
    # 6. Compute forces
        print('Computing forces...')
        Tf = time.time()
        if PTL.NB == 2:
            traj1 = PTL.listTrajectories[0]
            traj2 = PTL.listTrajectories[1]
            B0 = timeSeries['B']
            D3 = timeSeries['D3']
            dx = timeSeries['dx']
            F, dfLogF = PTL.forceCompute(traj1, traj2, B0, D3, dx)
            timeSeries['F'] = F
        
        print('OK! dT = {:.3f}'.format(time.time()-Tf))
            
            # Magnetization [A.m^-1]
            # M270
            # M = 0.74257*1.05*1600*(0.001991*B.^3+17.54*B.^2+153.4*B)./(B.^2+35.53*B+158.1)
            # M450
            # M = 1.05*1600*(0.001991*B.^3+17.54*B.^2+153.4*B)./(B.^2+35.53*B+158.1);
    
    # Save the tables !
        if PTL.NB == 2:
            timeSeries_DF = pd.DataFrame(timeSeries)
            timeSeriesFilePath = os.path.join(timeSeriesDataDir, f[:-4] + '_PY.csv')
            timeSeries_DF.to_csv(timeSeriesFilePath, sep = ';', index=False)
    
    
          
    # 7. Export the results
    
    print('\nTotal time:')
    print(time.time()-start)
    print('\n')
    return(PTL, timeSeries_DF, dfLogF)

#

In [6]:
mainDataDir = 'D://MagneticPincherData'
rawDataDir = os.path.join(mainDataDir, 'Raw')
depthoDir = os.path.join(rawDataDir, 'EtalonnageZ')
interDataDir = os.path.join(mainDataDir, 'Intermediate')
figureDir = os.path.join(mainDataDir, 'Figures')
timeSeriesDataDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//DataAnalysis//TimeSeriesData"

In [9]:
dates = '21.10.18'
manips = 1
wells = 1
cells = [15]
depthoNames = '21.10.18_M1_M270_100X_step20'

PTL, timeSeries_DF, dfLogF = main(dates, manips, wells, cells, depthoNames, 
                                  methodT = 'max_entropy', factorT = 0.80, redoAllSteps = False)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'




Analysis of file 1/1 : 21-10-18_M1_P1_C15_R40_disc20um.tif
Loading image and experimental data...
OK!
Pretreating the image...
OK!
Detecting all the bead objects...


C:\Users\JosephVermeil\Desktop\ActinCortexAnalysis\DataAnalysis\PythonCode\BeadTracker.py:1272: RuntimeWarning: divide by zero encountered in double_scalars
  Circ = (4 * np.pi * A) / (P * P)


OK! dT = 136.237
Tracking the beads of interest...
OK! dT = 30.171
Computing Z in traj  1...


C:\Users\JosephVermeil\Desktop\ActinCortexAnalysis\DataAnalysis\PythonCode\BeadTracker.py:1765: RuntimeWarning: divide by zero encountered in long_scalars
  if frequency%iS == 0 or init:


OK! dT = 5.754
Computing Z in traj  2...
OK! dT = 5.497
Computing distances...
OK!
Computing forces...
OK! dT = 0.037

Total time:
204.41728401184082


